In [1]:
import numpy as np
import pickle
import os
from tqdm.notebook import tqdm

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swmhau_network_functions import (
    swmhau_network_hyperparameter_search,
)

In [4]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [5]:
%run ../load_anno_mi.py

In [6]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime,speaker
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-10-04 00:00:13,-1
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-10-04 00:00:24,1
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-10-04 00:00:25,-1
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-10-04 00:00:34,1
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-10-04 00:00:34,-1


In [7]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

# swmhau Network

## Obtaining path by looking at post history

We can obtain a path by looking at the history of each post. Here we look at the last 10 posts (and pad with vectors of zeros if there are less than 10 posts) including the current post.

We only want to consider paths that correspond to a client's utterance as we want to model a change in mood at that time. Their history will still contain the therapist's utterances too.

In [8]:
features = ["timeline_index", "speaker"]
standardise_method = [None, None]
include_features_in_path = True
include_features_in_input = False

In [9]:
num_epochs = 100
dimensions = [15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (8, 4, 6)]
num_layers = [1]
ffn_hidden_dim_sizes = [[256, 256], [512, 512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [10]:
kwargs = {
    "num_epochs": num_epochs,
    "df": anno_mi,
    "id_column": "transcript_id",
    "label_column": "client_talk_type",
    "embeddings": sbert_embeddings,
    "y_data": y_data_client,
    "output_dim": output_dim_client,
    "dimensions": dimensions,
    "log_signature": True,
    "pooling": "signature",
    "swmhau_parameters": swmhau_parameters,
    "num_layers": num_layers,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "path_indices": client_index,
    "split_ids": client_transcript_id,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# w=5

In [11]:
size = 5

## UMAP

In [ ]:
(
    swmhau_network_umap_kfold_5,
    best_swmhau_network_umap_kfold_5,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
swmhau_network_umap_kfold_5.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_5

In [ ]:
best_swmhau_network_umap_kfold_5["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_5["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_5["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_5["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_5["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_5["recall_scores"]).mean(axis=0)

## Random Projections

In [ ]:
(
    swmhau_network_grp_kfold_5,
    best_swmhau_network_grp_kfold_5,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_grp_kfold_5.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_grp_kfold_5

In [ ]:
best_swmhau_network_grp_kfold_5["f1"].mean()

In [ ]:
best_swmhau_network_grp_kfold_5["precision"].mean()

In [ ]:
best_swmhau_network_grp_kfold_5["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_grp_kfold_5["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_5["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_5["recall_scores"]).mean(axis=0)

# w=11

In [ ]:
size = 11

## UMAP

In [ ]:
(
    swmhau_network_umap_kfold_11,
    best_swmhau_network_umap_kfold_11,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_umap_kfold_11.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_11

In [ ]:
best_swmhau_network_umap_kfold_11["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_11["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_11["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_11["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_11["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_11["recall_scores"]).mean(axis=0)

## Random Projections

In [ ]:
(
    swmhau_network_grp_kfold_11,
    best_swmhau_network_grp_kfold_11,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_grp_kfold_11.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_grp_kfold_11

In [ ]:
best_swmhau_network_grp_kfold_11["f1"].mean()

In [ ]:
best_swmhau_network_grp_kfold_11["precision"].mean()

In [ ]:
best_swmhau_network_grp_kfold_11["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_grp_kfold_11["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_11["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_11["recall_scores"]).mean(axis=0)

# w=20

In [ ]:
size = 20

## UMAP

In [ ]:
(
    swmhau_network_umap_kfold_20,
    best_swmhau_network_umap_kfold_20,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_umap_kfold_20.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_20

In [ ]:
best_swmhau_network_umap_kfold_20["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_20["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_20["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_20["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_20["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_20["recall_scores"]).mean(axis=0)

## Random Projections

In [ ]:
(
    swmhau_network_grp_kfold_20,
    best_swmhau_network_grp_kfold_20,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_grp_kfold_20.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_grp_kfold_20

In [ ]:
best_swmhau_network_grp_kfold_20["f1"].mean()

In [ ]:
best_swmhau_network_grp_kfold_20["precision"].mean()

In [ ]:
best_swmhau_network_grp_kfold_20["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_grp_kfold_20["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_20["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_20["recall_scores"]).mean(axis=0)

# w=35

In [ ]:
size = 35

## UMAP

In [ ]:
(
    swmhau_network_umap_kfold_35,
    best_swmhau_network_umap_kfold_35,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_umap_kfold_35.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_35

In [ ]:
best_swmhau_network_umap_kfold_35["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_35["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_35["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_35["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_35["recall_scores"]).mean(axis=0)

## Random Projections

In [ ]:
(
    swmhau_network_grp_kfold_35,
    best_swmhau_network_grp_kfold_35,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_grp_kfold_35.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_grp_kfold_35

In [ ]:
best_swmhau_network_grp_kfold_35["f1"].mean()

In [ ]:
best_swmhau_network_grp_kfold_35["precision"].mean()

In [ ]:
best_swmhau_network_grp_kfold_35["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_grp_kfold_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_35["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_35["recall_scores"]).mean(axis=0)

# w=80

In [ ]:
size = 80

## UMAP

In [ ]:
(
    swmhau_network_umap_kfold_80,
    best_swmhau_network_umap_kfold_80,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_umap_kfold_80.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_80

In [ ]:
best_swmhau_network_umap_kfold_80["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_80["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_80["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["recall_scores"]).mean(axis=0)

## Random Projections

In [ ]:
(
    swmhau_network_grp_kfold_80,
    best_swmhau_network_grp_kfold_80,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_grp_kfold_80.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_grp_kfold_80

In [ ]:
best_swmhau_network_grp_kfold_80["f1"].mean()

In [ ]:
best_swmhau_network_grp_kfold_80["precision"].mean()

In [ ]:
best_swmhau_network_grp_kfold_80["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_grp_kfold_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_80["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_80["recall_scores"]).mean(axis=0)

## w=110

In [ ]:
size = 110

## UMAP

In [ ]:
(
    swmhau_network_umap_kfold_110,
    best_swmhau_network_umap_kfold_110,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_umap_kfold_110.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_110

In [ ]:
best_swmhau_network_umap_kfold_110["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_110["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_110["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_110["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_110["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_110["recall_scores"]).mean(axis=0)

## Random Projections

In [ ]:
(
    swmhau_network_grp_kfold_110,
    best_swmhau_network_grp_kfold_110,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_grp_kfold_5.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_grp_kfold_20

In [62]:
best_swmhau_network_grp_kfold_20["f1"].mean()

0.48470926457915864

In [63]:
best_swmhau_network_grp_kfold_20["precision"].mean()

0.48288163634738

In [64]:
best_swmhau_network_grp_kfold_20["recall"].mean()

0.48735577008635894

In [65]:
np.stack(best_swmhau_network_grp_kfold_20["f1_scores"]).mean(axis=0)

array([0.75505381, 0.39767772, 0.30139626])

In [66]:
np.stack(best_swmhau_network_grp_kfold_20["precision_scores"]).mean(axis=0)

array([0.76263008, 0.39263368, 0.29338114])

In [67]:
np.stack(best_swmhau_network_grp_kfold_20["recall_scores"]).mean(axis=0)

array([0.74802391, 0.40374037, 0.31030303])

# w=35

In [68]:
size = 35

## UMAP

In [69]:
(
    swmhau_network_umap_kfold_35,
    best_swmhau_network_umap_kfold_35,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_35_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_35_kfold_best_model.csv


In [70]:
swmhau_network_umap_kfold_35.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2005997/548979897.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_35.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.593934   
                                                                         0.0005         0.627427   
                                                                         0.0010         0.632950   
                                                          (512, 512)     0.0001         0.599617   
                                                                         0.0005         0.632727   
                                                                         0.0010         0.632024   
           12              3         10        1          (256, 256)     0.0001         0.588985   
                                                                         0.0005         0.619891   
                                                                         0.0010         0.625798   
                                                          (512, 512)     0.0001         0.591028   
                                                                         0.0005         0.613027   
                                                                         0.0010         0.631450   

                                                                                              f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.390413   
                                                                         0.0005         0.507085   
                                                                         0.0010         0.518019   
                                                          (512, 512)     0.0001         0.415420   
                                                                         0.0005         0.511531   
                                                                         0.0010         0.513059   
           12              3         10        1          (256, 256)     0.0001         0.422396   
                                                                         0.0005         0.494118   
                                                                         0.0010         0.510472   
                                                          (512, 512)     0.0001         0.440987   
                                                                         0.0005         0.490600   
                                                                         0.0010         0.512296   

                                                                                        precision  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate              
15         8               4         6         1          (256, 256)     0.0001          0.405132   
                                                                         0.0005          0.500641   
                                                                         0.0010          0.511158   
                                                          (512, 512)     0.0001          0.427870   
                                                                         0.0005          0.504813   
                                                                         0.0010          0.507141   
           12              3         10        1          (256, 256)     0.0001          0.426169   
                                                                         0.0005          0.487869   
                                                                         0.0010          0.503684   
                                                          (512, 512)     0.0001          0.442288   
                                                                         0

In [71]:
best_swmhau_network_umap_kfold_35

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.615900,0.470222,"[0.7566293459045375, 0.4083640836408364, 0.245...",0.467258,"[0.7711711711711712, 0.4058679706601467, 0.224...",0.474809,"[0.7426257952573742, 0.41089108910891087, 0.27...",None,0.675979,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.620115,0.505004,"[0.7494343038165636, 0.4261224489795919, 0.339...",0.496610,"[0.783349101229896, 0.42164781906300486, 0.284...",0.523009,"[0.7183342972816658, 0.4306930693069307, 0.42]",None,0.669236,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.618966,0.501638,"[0.7480777928539123, 0.42376856919468336, 0.33...",0.493887,"[0.7814173228346457, 0.4026745913818722, 0.297...",0.514281,"[0.7174667437825333, 0.4471947194719472, 0.378...",None,0.665703,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [72]:
best_swmhau_network_umap_kfold_35["f1"].mean()

0.4922879075088247

In [73]:
best_swmhau_network_umap_kfold_35["precision"].mean()

0.48591839085193883

In [74]:
best_swmhau_network_umap_kfold_35["recall"].mean()

0.5040329581444746

In [75]:
np.stack(best_swmhau_network_umap_kfold_35["f1_scores"]).mean(axis=0)

array([0.75138048, 0.41941837, 0.30606487])

In [76]:
np.stack(best_swmhau_network_umap_kfold_35["precision_scores"]).mean(axis=0)

array([0.77864587, 0.41006346, 0.26904585])

In [77]:
np.stack(best_swmhau_network_umap_kfold_35["recall_scores"]).mean(axis=0)

array([0.72614228, 0.42959296, 0.35636364])

## Random Projections

In [78]:
(
    swmhau_network_grp_kfold_35,
    best_swmhau_network_grp_kfold_35,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_35_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_35_kfold_best_model.csv


In [79]:
swmhau_network_grp_kfold_35.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2005997/2891164155.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_grp_kfold_35.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.598851   
                                                                                      0.0005         0.634930   
                                                                                      0.0010         0.639847   
                                                                         0.2          0.0001         0.614049   
                                                                                      0.0005         0.625415   
                                                                                      0.0010         0.626309   
                                                          (512, 512)     0.1          0.0001         0.619860   
                                                                                      0.0005         0.629119   
                                                                                      0.0010         0.635568   
                                                                         0.2          0.0001         0.608748   
                                                                                      0.0005         0.631418   
                                                                                      0.0010         0.634738   
           12              3         10        1          (256, 256)     0.1          0.0001         0.604151   
                                                                                      0.0005         0.623499   
                                                                                      0.0010         0.639974   
                                                                         0.2          0.0001         0.609706   
                                                                                      0.0005         0.624840   
                                                                                      0.0010         0.637548   
                                                          (512, 512)     0.1          0.0001         0.609195   
                                                                                      0.0005         0.629502   
                                                                                      0.0010         0.627969   
                                                                         0.2          0.0001         0.611622   
                                                                                      0.0005         0.620243   
                                                                                      0.0010         0.635249   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.445935   
                                                                                      0.0005         0.520999   
                                                                                      0.0010         0.521835   
                                                                         0.2          0.0001         0.460678   
                                                                                      0.0005         0.512821   
                                                                                      0.0010         0.514044   
                                                          (512, 512)     0.1          0.0001         0.475734   
                                                                                      0.0005         0.515756   
                                

In [80]:
best_swmhau_network_grp_kfold_35

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.591379,0.466577,"[0.7259013425856087, 0.3855243722304284, 0.288...",0.465068,"[0.7587511825922422, 0.34893048128342247, 0.28...",0.471854,"[0.6957779063042221, 0.4306930693069307, 0.289...",None,0.677906,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.612644,0.467092,"[0.7468224981738496, 0.3958171959721146, 0.258...",0.470193,"[0.7546501328609388, 0.372992700729927, 0.2829...",0.466318,"[0.7391555812608445, 0.42161716171617164, 0.23...",None,0.672929,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.623563,0.492072,"[0.7555687555687556, 0.4196670538133953, 0.300...",0.488943,"[0.7765567765567766, 0.3953318745441284, 0.294...",0.496718,"[0.7356853672643147, 0.4471947194719472, 0.307...",None,0.681599,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [81]:
best_swmhau_network_grp_kfold_35["f1"].mean()

0.4752470078906279

In [82]:
best_swmhau_network_grp_kfold_35["precision"].mean()

0.474734670614678

In [83]:
best_swmhau_network_grp_kfold_35["recall"].mean()

0.4782965844299873

In [84]:
np.stack(best_swmhau_network_grp_kfold_35["f1_scores"]).mean(axis=0)

array([0.7427642 , 0.40033621, 0.28264062])

In [85]:
np.stack(best_swmhau_network_grp_kfold_35["precision_scores"]).mean(axis=0)

array([0.76331936, 0.37241835, 0.2884663 ])

In [86]:
np.stack(best_swmhau_network_grp_kfold_35["recall_scores"]).mean(axis=0)

array([0.72353962, 0.43316832, 0.27818182])

# w=80

In [87]:
size = 80

## UMAP

In [ ]:
(
    swmhau_network_umap_kfold_80,
    best_swmhau_network_umap_kfold_80,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_umap_kfold_80.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_80

In [ ]:
best_swmhau_network_umap_kfold_80["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_80["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_80["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["recall_scores"]).mean(axis=0)

## Random Projections

In [ ]:
(
    swmhau_network_grp_kfold_80,
    best_swmhau_network_grp_kfold_80,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_grp_kfold_80.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_grp_kfold_80

In [ ]:
best_swmhau_network_grp_kfold_80["f1"].mean()

In [ ]:
best_swmhau_network_grp_kfold_80["precision"].mean()

In [ ]:
best_swmhau_network_grp_kfold_80["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_grp_kfold_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_80["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_80["recall_scores"]).mean(axis=0)

## w=110

In [ ]:
size = 110

## UMAP

In [ ]:
(
    swmhau_network_umap_kfold_110,
    best_swmhau_network_umap_kfold_110,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_umap_kfold_110.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_110

In [ ]:
best_swmhau_network_umap_kfold_110["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_110["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_110["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_110["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_110["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_110["recall_scores"]).mean(axis=0)

## Random Projections

In [ ]:
(
    swmhau_network_grp_kfold_110,
    best_swmhau_network_grp_kfold_110,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_grp_kfold_110.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_grp_kfold_110

In [ ]:
best_swmhau_network_grp_kfold_110["f1"].mean()

In [ ]:
best_swmhau_network_grp_kfold_110["precision"].mean()

In [ ]:
best_swmhau_network_grp_kfold_110["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_grp_kfold_110["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_110["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_110["recall_scores"]).mean(axis=0)